# Wikimedia data
Wikimedia Downloads: Analytics Datasets Info about Pageviews, mediacounts and unique devices:
Pageviews since may 2015:
https://dumps.wikimedia.org/other/pageviews/[YEAR]/[YEAR]-[2-DIGIT-MONTH]/projectviews-YYYYMMDD-HH0000

In [ ]:
from datetime import datetime, timedelta
from itertools import chain
from functools import reduce
import requests
import re
import os

def time_range(init, end):
    while init<end:
        yield init
        init+=timedelta(hours=1)   

def explode_time(date):
    return date, {
        'year': date.year,
        'month': date.month,
        'day': date.day,
        'hour': date.hour,
    }

def gen_dateurl(x):
    ( date, exploded_date ) = x
    ( url_template ) = (
        'https://dumps.wikimedia.org/other/pageviews/'
        '{year}/{year}-{month:02d}/'
        'projectviews-{year}{month:02d}{day:02d}-{hour:02d}0000'
    )
    return date, url_template.format(**exploded_date)

def explode_data_lines(line):
    m = DATALINE_REGEX.match(line)
    return {
        'project': m.group('project'),
        'hits': m.group('hits'),
    }

def explode_data_files(dateurl):
    ( date, url ) = dateurl
    
    file_name = 'cache/' + url.replace('/','_')
    
    if not os.path.exists(file_name):
        os.makedirs('cache', exist_ok=True)
        with open(file_name, 'w') as f:
            f.write(requests.get(url).text)
        f.closed
    
    with open(file_name, 'r') as f:
        file_content = f.read()
    f.closed

    match_iter = DATALINE_REGEX.finditer(file_content)

    print(list(re.finditer(r"(?P<project>en|es) - (?P<hits>\d+)", file_content)))
    
    for match in match_iter:
        data['project'] = match.group('project')
        data['hits'] = match.group('hits')
        data['date'] = date
        yield data
        
        
def filter_wikipedia_en(data):
    return data['project'] == 'en'

DATALINE_REGEX = re.compile(r"/(?P<project>en) - (?P<hits>\d+)/mg")

exploded_dates = map(explode_time, time_range(datetime(2016,10,31), datetime(2017,10,31)))

dateurls = map(gen_dateurl, exploded_dates)

data_generators = map(explode_data_files, dateurls)

data_generator = reduce(chain, data_generators)

for item in data_generator:
    pass
    #print(item['date'], data['hits'])
